<a href="https://colab.research.google.com/github/go23man/study_project/blob/master/project/edge_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import base64
import html
import io
import time

from IPython.display import display, Javascript
from google.colab.output import eval_js
import numpy as np
from PIL import Image
import cv2

def start_input():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 512, 512);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);
      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();
      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 512; //video.videoWidth;
      captureCanvas.height = 512; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function takePhoto(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }
      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def take_photo(label, img_data):
  data = eval_js('takePhoto("{}", "{}")'.format(label, img_data))
  return data

In [ ]:
start_input()
label_html = 'Capturing...'
img_data = ''

while True:
  capture_start = time.time()
  js_reply = take_photo(label_html, img_data)
  capture_end = time.time()
  if not js_reply:
    break

In [ ]:
def js_reply_to_image(js_reply):
    """
    input: 
          js_reply: JavaScript object, contain image from webcam
    output: 
          image_array: image array RGB size 512 x 512 from webcam
    """
    jpeg_bytes = base64.b64decode(js_reply['img'].split(',')[1])
    image_PIL = Image.open(io.BytesIO(jpeg_bytes))
    image_array = np.array(image_PIL)

    return image_array

In [ ]:
from google.colab import drive
from google.colab import files
from google.colab import auth

drive.mount('/content/gdrive')
!ls /content/gdrive/

auth.authenticate_user()

project_id = 'vocal-oarlock-289106'
!gcloud config set project {project_id}
!gsutil ls

bucket_name = 'hanium_bucket_201007'

In [ ]:
import os
from scipy import ndimage
from datetime import datetime
from matplotlib import pyplot as plt

fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter('/content/gdrive/My Drive/record_files2/output.avi', fourcc, 30, (512, 512))

start_input()
label_html = 'Capturing...'
img_data = ''
count = 0 

while True:
    js_reply = take_photo(label_html, img_data)
    if not js_reply:
        break

    image = js_reply_to_image(js_reply)
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    writer.write(image)

    now = datetime.now()
    snow = str(now.year) + '_' + str(now.month) + '_' + str(now.day) + '_' + str(now.hour + 9) + '_' + str(now.minute) + '_' + str(now.second)

    cv2.imwrite('/content/gdrive/My Drive/record_files2/color_images/' + snow + '.png', image)
    cv2.imwrite('/content/gdrive/My Drive/record_files2/gray_images/' + snow + '(2).png', gray_image)
    filename = snow + '.png'
    filename2 = snow + '(2).png'
    !gsutil cp -r /content/gdrive/My\ Drive/record_files2/color_images/{filename} gs://{bucket_name}/record_files2/color_images/
    !gsutil cp -r /content/gdrive/My\ Drive/record_files2/gray_images/{filename2} gs://{bucket_name}/record_files2/gray_images/

writer.release()
cv2.destroyAllWindows()
!gsutil cp -r /content/gdrive/My\ Drive/record_files2/output.avi gs://{bucket_name}/record_files2/ # 구글 드라이브에 있는 파일을 구글클라우드에 저장

image_folder = '/content/gdrive/My Drive/record_files2/gray_images'
image_folder2 = '/content/gdrive/My Drive/record_files2/ed_images'
video_name = '/content/gdrive/My Drive/record_files2/' + snow + '.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
num = 0

# input image (gray)
for image in images:
    inimg = plt.imread(os.path.join(image_folder, image))
    inimg = np.uint8(255*inimg)

    # =============================================================================
    # # edge detection
    # =============================================================================
    x,y = inimg.shape
    edge = np.zeros(256)
    sobel1 = 1/4*np.array([[1,0,-1],
                           [2,0,-2],
                           [1,0,-1]])

    sobel2 = 1/4*np.array([[-1,-2,-1],
                           [0,0,0],
                           [1,2,1]])

    inimg_x = ndimage.convolve(inimg[:,:], sobel1)
    inimg_y = ndimage.convolve(inimg[:,:], sobel2)

    #x gradient 
    inimg_x = np.uint8(inimg_x)

    #y gradient 
    inimg_y = np.uint8(inimg_y)

    # sobel_inimg = abs(inimg_x) + abs(inimg_y)
    sobel_inimg = (inimg_x**2 + inimg_y**2)**0.5
    sobel_outimg = np.zeros([x,y])
    threshold = 5   

    for i in range(x):
        for j in range(y):
            if (sobel_inimg[i,j] >= threshold):
                sobel_outimg[i,j] = 0
            else:
                sobel_outimg[i,j] = 255

    sobel_outimg = np.float32(sobel_outimg)
    ed_name = image_folder2 + '/' + str(num) + '.png'
    cv2.imwrite(ed_name, sobel_outimg)
    num += 1

!gsutil -m cp -r /content/gdrive/My\ Drive/record_files2/ed_images/ gs://{bucket_name}/record_files2/

images2 = [img for img in os.listdir(image_folder2) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder2, images2[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'XVID'), 1, (width,height))

for image in images2:
    video.write(cv2.imread(os.path.join(image_folder2, image)))

!gsutil cp -r /content/gdrive/My\ Drive/record_files2/{snow}.avi gs://{bucket_name}/record_files2/

cv2.destroyAllWindows()
video.release()